# Convolutional Neural Network

### Importing the libraries

In [1]:
# !pip install --upgrade tensorflow keras Pillow

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2021-08-26 16:12:58.539388: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-26 16:12:58.539417: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

2021-08-26 16:13:01.114354: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-26 16:13:01.115275: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-26 16:13:01.115402: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-08-26 16:13:01.115495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-08-26 16:13:01.115592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation = 'relu'))
cnn.add(tf.keras.layers.Dense(units=128, activation = 'relu'))

### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [22]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [23]:
# !pip install SciPy
cnn.fit(x = training_set, validation_data = test_set, epochs = 10)
# limpei o output pq é feio

Epoch 1/10
250/250 [==============================] - 30s 118ms/step - loss: 0.3779 - accuracy: 0.8300 - val_loss: 0.4576 - val_accuracy: 0.7975
Epoch 2/10
250/250 [==============================] - 33s 132ms/step - loss: 0.3612 - accuracy: 0.8394 - val_loss: 0.4937 - val_accuracy: 0.7840
Epoch 3/10
250/250 [==============================] - 36s 144ms/step - loss: 0.3362 - accuracy: 0.8480 - val_loss: 0.4961 - val_accuracy: 0.7875
Epoch 4/10
250/250 [==============================] - 37s 149ms/step - loss: 0.3268 - accuracy: 0.8562 - val_loss: 0.4746 - val_accuracy: 0.7985
Epoch 5/10
250/250 [==============================] - 38s 151ms/step - loss: 0.3080 - accuracy: 0.8696 - val_loss: 0.5614 - val_accuracy: 0.7625
Epoch 6/10
250/250 [==============================] - 35s 142ms/step - loss: 0.2975 - accuracy: 0.8699 - val_loss: 0.4851 - val_accuracy: 0.7985
Epoch 7/10
250/250 [==============================] - 33s 133ms/step - loss: 0.2840 - accuracy: 0.8832 - val_loss: 0.5624 - val_ac

## Part 4 - Making a single prediction

In [24]:
import numpy as np
from keras.preprocessing import image
for i in range(1,13):
    test_image = image.load_img(f'dataset/single_prediction/cat_or_dog_{i}.jpg', target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = cnn.predict(test_image)
    training_set.class_indices
    if result[0][0] == 1:
        prediction = 'dog'
    else:
        prediction = 'cat'
    print(f"{i} -> {prediction}")

1 -> dog
2 -> cat
3 -> dog
4 -> dog
5 -> dog
6 -> cat
7 -> cat
8 -> dog
9 -> dog
10 -> dog
11 -> dog
12 -> cat


## Conclusao

Provavelmente os dados se encontram desbalanceados de maneira que se mostra que o modelo esta viciado